# Clasificación de documentos **WORD2VEC**
Cundo se quiere realizar una clasificación de documentos mediante Word2Vec el primer paso es la construcción del modelo y con ello la del **Corpus de entrenamiento**

### Eliminación de StopWords y caracteres especiales:
Comenzaremos utilizando un archivo llamado *stop_words.txt* en el cual se tiene una lista de palabras que no aportan valor al corpus de entrenamiento por lo que son desechadas. Igualmente removemos todos los caracteres no alfanuméricos y los dígitos para contener unicamente las palabras.

In [9]:
from unidecode import unidecode
import re
import nltk
from nltk import *
from gensim.models import Word2Vec
from gensim.models.keyedvectors import Word2VecKeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy


LISTA_NO_ALFANUMERICOS = ["!", "@", "#", "$", "%", "&", "*", "(", ")", "-", "", "+", "=", "{", "}", "[", "]", "|", "'", "\\", ";", ":", ",", ".", "<", ">", "/", "?", "¡", "¿", "¨", "´", "¨", "~", "¬", "`", "^", "¦", "ª", "º", "«", "»", "©", "®", "™", "§", "±", "µ", "÷", "×", "``", "\\d", "-"]

def tokenizar(archivo):
    with open(archivo, "r", encoding="utf-8") as file:
        texto = file.read()

    # Tokenizar el texto en frases
    frases = re.sub("\\d", "", unidecode(texto))
    frases = re.sub("\\.", "", frases)
    frases = nltk.sent_tokenize(frases)

    # Tokenizar cada frase en palabras
    corpus = [word_tokenize(frase) for frase in frases]

    stop_words = open("stop_words.txt", encoding='utf-8')
    stop_words = stop_words.read()
    stop_words = unidecode(stop_words).split("\n")

    res = []

    for i in corpus:
        res1 = []
        for j in i:
            if j.lower() not in stop_words and j not in LISTA_NO_ALFANUMERICOS and len(j) > 1:
                res1.append(j.lower())
    res.append(res1)
    return res

En esta función podremos observar como realizamos la tokenización de las palabras, es decir, adaptar los textos para que se conviertan en vectores de palabras que después nos servirán para entrenar el modelo.

### Creación y entrenamiento del modelo:
A continuación realizaremos las funciones que se encargarán de leer los archivos alojados en la carpeta corpus, de tal manera dichos archivos se añadirán al corpus y se guardarán con su título, el cual representa la temática

Por ejemplo:
`filosofía.txt` - archivo que contiene un texto con temática filosófica.

In [10]:
def add_text_to_corpus(corpus,tematica,tematicas):
    new_corpus = tokenizar("corpus/"+tematica+".txt")
    corpus += new_corpus
    tematicas.append(tematica)
    return corpus, tematicas


def train_word2vec(corpus):
    model = Word2Vec(
        corpus,
        vector_size=100,
        window=5,
        min_count=1,
        workers=4
    )
    model.train(corpus, total_examples=len(corpus), epochs=model.epochs)
    return model

De esta manera solo nos hará falta llamar a las funciones en el método main para evitar grandes volúmenes de código y hacer este mucho mas legible e intuitibo.

### Creación de *Embeddings*
Los embeddings son los vectores de palabras que se crean a partir del corpus de entrenamiento y el modelo, estos representan la semántica de las palabras en función de su contexto en el corpus y se genera un vector de números reales.
Estos embeddings nos permiten determinar la diferencia entre palabras y encontrar palabras mas o menos similares.

In [11]:

def embeddings(corpus, vectores):
    corpus_embeddings = []
    for c in corpus:
        # Obtener el embedding de cada palabra en la frase y promediarlos
        embeddings = [vectores[word] for word in c if word in vectores]
        if embeddings:
            sentence_embedding = numpy.mean(embeddings, axis=0)
            corpus_embeddings.append(sentence_embedding)
    return corpus_embeddings


En este fragmento del codigo se devolverán los emedings pasándole como parámetros el corpues (puede ser entrenamiento o test) y el modelo entrenado.

## Función main:
Es aquí donde se realizan las llamadas a función y se ejecuta finalmente el programa.

In [12]:
def main():
    # Carpeta donde se encuentran los archivos de texto
    folder_path = "corpus"

    # Corpus inicial y temáticas
    corpus = []
    tematicas = []

    # Añadir textos al corpus
    for filename in os.listdir(folder_path):
      if filename.endswith(".txt"):
        tematica = filename.split(".txt")[0]
        corpus, tematicas = add_text_to_corpus(corpus, tematica, tematicas)


    # Entrenar el modelo Word2Vec
    model = train_word2vec(corpus)

    # Obtener representaciones de embeddings
    test_file = "test/test3.txt"
    test_corpus = tokenizar(test_file)
    test_embeddings = embeddings(test_corpus, model.wv)
    total_embeddings = embeddings(corpus, model.wv)

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train = total_embeddings
    y_train = tematicas

    # Entrenar un modelo de clasificación
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    resultado_clasificacion = model.predict(test_embeddings)
    print("El resultado de la clasificación fue: " + resultado_clasificacion[0])

    # Evaluar el rendimiento del modelo

Podemos observar como hemos instanciado algunas variables como corpus o tematicas que usaremos durante la ejecución y que incluse se utilizan de forma recursiva para modificar sus valores.
Hay que tener en cuenta que el texto test no se ha configurado para su lectura automática debido a que puede haber mas de un texto en la carpeta, por lo que debe ajustarse de forma manual cual de los textos de prueba se quiere clasificar.

Para ejecutar el código completo solamente debemos llamar a la función `main`.

In [13]:
main()

El resultado de la clasificación fue: fotografía
